In [5]:
import cv2
import numpy as np
import os

# File paths
weights_path = r"C:\Users\deeps\Downloads\obj3.weights"
config_path = r"C:\Users\deeps\Downloads\obj3.cfg"
names_path = r"C:\Users\deeps\Downloads\coco.names"
output_video_path = r"C:\Users\deeps\Downloads\output_video.avi"

# Verify file paths
if not os.path.exists(weights_path):
    print(f"Error: Weights file not found at {weights_path}")
    exit()
if not os.path.exists(config_path):
    print(f"Error: Config file not found at {config_path}")
    exit()
if not os.path.exists(names_path):
    print(f"Error: Class names file not found at {names_path}")
    exit()

# Load YOLO model
net = cv2.dnn.readNet(weights_path, config_path)

# Enable GPU support if available
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Load class names
with open(names_path, "r") as f:
    class_names = [line.strip() for line in f.readlines()]

# Get output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture video.")
        break

    # Get frame dimensions
    height, width = frame.shape[:2]

    # Create a blob and set it as input to the network
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Perform forward pass
    detections = net.forward(output_layers)

    # Initialize lists for detections
    boxes, confidences, class_ids = [], [], []

    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = int(np.argmax(scores))
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                # Get bounding box coordinates
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype("int")
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to filter boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw detections
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = f"{class_names[class_ids[i]]}: {confidences[i]:.2f}"
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.namedWindow("Object Detection", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Object Detection", 1000, 720)
    cv2.imshow("Object Detection", frame)

    # Break the loop if 'a' is pressed
    if cv2.waitKey(1) & 0xFF == ord('a'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()
